In [2]:
pip install pyfpgrowth

  Created wheel for pyfpgrowth: filename=pyfpgrowth-1.0-py2.py3-none-any.whl size=5477 sha256=ed2e44ee96456f448c05ce59341ed69e103d55aa8b7b2829b01c500a027233bd
  Stored in directory: c:\users\android\appdata\local\pip\cache\wheels\29\e2\a7\ccb34c35e2c6737d3f09b53784f4323b7ff556a5f02096c70d
Successfully built pyfpgrowth
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as p
import numpy as np
import statistics
import pyfpgrowth

transactions = [
    ['1', '2', '5'],
    ['2', '4'],
    ['2', '3'],
    ['1', '2', '4'],
    ['1', '3'],
    ['2', '3'],
    ['1', '3'],
    ['1', '2', '3', '5'],
    ['1', '2', '3']
]

print(transactions)

[['1', '2', '5'], ['2', '4'], ['2', '3'], ['1', '2', '4'], ['1', '3'], ['2', '3'], ['1', '3'], ['1', '2', '3', '5'], ['1', '2', '3']]


In [7]:
FrequentPatterns = pyfpgrowth.find_frequent_patterns(transactions = transactions, support_threshold = 0.5)
print("Frequent Patterns: \n", FrequentPatterns)

Rules = pyfpgrowth.generate_association_rules(patterns = FrequentPatterns, confidence_threshold = 0.5)
print("Rules: \n", Rules)

Frequent Patterns: 
 {('5',): 2, ('1', '5'): 2, ('2', '5'): 2, ('3', '5'): 1, ('1', '2', '5'): 2, ('1', '3', '5'): 1, ('2', '3', '5'): 1, ('1', '2', '3', '5'): 1, ('4',): 2, ('2', '4'): 2, ('1', '4'): 1, ('1', '2', '4'): 1, ('1',): 6, ('1', '2'): 4, ('2', '3'): 4, ('1', '2', '3'): 2, ('1', '3'): 4, ('2',): 7}
Rules: 
 {('5',): (('1', '2', '3'), 0.5), ('1', '2'): (('3',), 0.5), ('1', '5'): (('2', '3'), 0.5), ('2', '5'): (('1', '3'), 0.5), ('3', '5'): (('1', '2'), 1.0), ('1', '2', '3'): (('5',), 0.5), ('1', '2', '5'): (('3',), 0.5), ('1', '3', '5'): (('2',), 1.0), ('2', '3', '5'): (('1',), 1.0), ('4',): (('1', '2'), 0.5), ('1', '4'): (('2',), 1.0), ('2', '4'): (('1',), 0.5), ('1',): (('3',), 0.6666666666666666), ('2',): (('3',), 0.5714285714285714), ('1', '3'): (('2',), 0.5), ('2', '3'): (('1',), 0.5)}


In [8]:
import pandas as pd
import pyfpgrowth
import numpy as np
import statistics

In [22]:
data = pd.read_csv("apriory_data.csv")
print(data)

min = 2
dataset = {}

for i in data.values.tolist():
    dataset[i[0]] = i[1].split(',')

    
transactions = []
for i in dataset.values():
    transactions.append(i)
patterns = pyfpgrowth.find_frequent_patterns(transactions, 2)
dpatterns = pd.DataFrame(patterns.items(), columns = ['ID', "List"])
print(dpatterns)

# Frequent Elements

m = 0
for i in patterns.keys():
    if len(i) > m:
        m = len(i)

frequent_items = {}
for i, j in patterns.items():
    if len(i) == m:
        frequent_items[i] = j

freqItem = pd.DataFrame(frequent_items.items(), columns = ['ID', "List"])
print(freqItem)

# Define the create node function to create a new node in the FP tree

def create_node(item, count, parent):
    node = {'item': item, 'count': count, 'parent': parent, 'children': {}}
    return node

# Define the update node function to update an existing node in the FP tree
def update_node(node, count):
    node['count'] += count

#Defind the main function to create the FP tree
def create_fp_tree(dataset):
    item_counts = []
    for tid, itemset in dataset.items():
        for item in itemset:
            if item in item_counts:
                item_counts[item] += 1
            else:
                item_counts[item] = 1
        
        sorted_items = [item for item, count in sorted(item_counts.itmes(), key=lambda x: x[1], reverse = True)]
        root = create_node(None, 0, None)
        
        # Traverse the dataset and update the FP tree
        for tid, itemset in dataset.items():
            itemset = [item for item in sorted_items if item in itemset]
            current_node = root
            for item in itemset:
                if(item in current_node["children"]):
                    child_node = current_node["children"][item]
                else:
                    child_node = create_node(item, 0, current_node)
                    current_node["children"][itme] = child_node
                update_node(child_node, 1)
                current_node = child_node
        
        return root
    

dataset = {
    0: ['I1', 'I2', 'I5'],
    1: ['I2','I4'],
    2: ['I2', 'I3'],
    3: ['I1', 'I2', 'I4'],
    4: ['I1', 'I3'],
    5: ['I2', 'I3'],
    6: ['I1', 'I3'],
    7: ['I1', 'I2', 'I3', 'I5'],
    8: ['I1', 'I2', 'I3']
}

fp_tree = create_fp_tree(dataset)

# Define the print tree function to prin the FP tree in a tree like structure

def print_tree(node, indent = 0):
    print("  "*indent + str(node['item']) + ' (' + str(node['count']) + ' (')
    for child_node in node['children'].value():
        print_tree(child_node, indenet + 1)
        
print(fp_tree)

    TID            List
0  T100      I1, I2, I5
1  T200          I2, I4
2  T300          I2, I3
3  T400      I1, I2, I4
4  T500          I1, I3
5  T600          I2, I3
6  T700          I1, I3
7  T800  I1, I2, I3, I5
8  T900      I1, I2, I3
                ID  List
0           ( I5,)     2
1       ( I2,  I5)     2
2        ( I5, I1)     2
3   ( I2,  I5, I1)     2
4           ( I4,)     2
5            (I2,)     3
6        ( I3, I2)     2
7           ( I2,)     4
8        ( I2, I1)     4
9       ( I2,  I3)     2
10  ( I2,  I3, I1)     2
11           (I1,)     6
12          ( I3,)     6
13       ( I3, I1)     4
               ID  List
0  ( I2,  I5, I1)     2
1  ( I2,  I3, I1)     2


TypeError: list indices must be integers or slices, not str